#Steam API key : XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
(My) Steam ID : 76561198048235598
"Sid Meier's Civilization VI" Appid = 289070

PLAYER INFO :
https://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key=XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX&steamids=76561198048235598


ACHIEVEMENTS INFO : 
https://api.steampowered.com/ISteamUserStats/GetPlayerAchievements/v0001/?appid=289070&key=XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX&steamid=76561198048235598

Metadata:
https://api.steampowered.com/ISteamUserStats/GetSchemaForGame/v2/?key=XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX&appid=289070

In [1]:
import requests
import pandas as pd
from datetime import datetime
import numpy as np

In [2]:
url_ach = 'https://api.steampowered.com/ISteamUserStats/GetSchemaForGame/v2/?key=XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX&appid=289070'
url_me = 'https://api.steampowered.com/ISteamUserStats/GetPlayerAchievements/v0001/?appid=289070&key=XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX&steamid=76561198048235598'

In [3]:
achievement_info = requests.get(url_ach)
achievement_info = achievement_info.json()
general_achievements = achievement_info['game']['availableGameStats']['achievements']

In [4]:
df_general = pd.DataFrame(columns = ['game_name', 'name', 'description'])
data = []

for i in range (len(general_achievements)):
    data.append([general_achievements[i]['name'], general_achievements[i]['displayName'],
                 general_achievements[i]['description']])

    
for j in range(len(data)):
    df_general.loc[len(df_general)] = data[j]

In [5]:
player_info = requests.get(url_me)
player_info = player_info.json()
player_info = player_info['playerstats']['achievements']

In [6]:
df_player = pd.DataFrame(columns = ['game_name', 'achieved'])
data = []


for k in range (len(player_info)):
    data.append([player_info[k]['apiname'], player_info[k]['achieved']])

    
for l in range(len(data)):
    df_player.loc[len(df_player)] = data[l]

In [7]:
df_merged = pd.merge(df_general, df_player, on ='game_name', how= 'outer')

In [8]:
for i in range(len(df_merged)):
    if df_merged.loc[i, 'achieved'] == 1:
        df_merged.loc[i, 'achieved'] = True
    else:
        df_merged.loc[i, 'achieved'] = False
        
df_merged = df_merged.drop(columns = ['game_name'])

In [9]:
leader_win = df_merged[(df_merged['description'].str.contains('Win a game as')) | (df_merged['description'].str\
                                                                                   .contains('Win a regular game as'))]

leader_win.reset_index(drop = True, inplace = True)

In [10]:
leaders_temp = []
for i in range (len(leader_win)):
    leaders_temp.append(leader_win['description'][i])
    
leaders = []

for i in range(len(leaders_temp)):
    
    if 'Win a regular game as' in leaders_temp[i]:
        leaders.append(leaders_temp[i].split('Win a regular game as ')[1])
    else:
        leaders.append(leaders_temp[i].split('Win a game as ')[1])

for i in range(len(leaders)):
    leaders[i] = leaders[i].strip('.')
    


leader_win.insert(0, 'Leader', leaders)

In [11]:
guides = pd.read_csv('guides.csv')

In [12]:
leaders_guide = []
for i in range(len(guides)):
    leaders_guide.append(guides['Leader'][i])

In [13]:
#Managing the differences in name from Steam documentation and the guides

leader_title = {'Peter' : 'Peter the Great', 'Wilhelmina' : 'Queen Wilhelmina', 'Seondeok' : 'Queen Seondeok'}

for key, values in leader_title.items():
    filter_condition = (guides['Leader'].str.contains(key))
    guides.loc[filter_condition, 'Leader'] = values
    

In [14]:
ach_guide = pd.merge(guides, leader_win[['Leader', 'achieved']], on = 'Leader', how = 'outer')

In [15]:
columns_to_modify = ['Culture', 'Diplomacy', 'Domination', 'Religion', 'Science']


for column in columns_to_modify:
    ach_guide[column] = ach_guide[column].apply(lambda x: int(x) if pd.notna(x) else x)

In [16]:
#Managing the multiple instances of the same leader with different names in guides and in steam documentation

doppelganger = ['Catherine', 'Suleiman', 'Saladin', 'Cleopatra', 'Qin', 'Eleanor', 'Theodore']

for i in doppelganger:

    row_to_copy = list(ach_guide[(ach_guide['Leader'].str.contains(i)) & (ach_guide['achieved'].notna())].index)[0]
    rows_to_replace = list(ach_guide[ach_guide['Leader'].str.contains(i)].index)
    
    if i == 'Theodore':
        rows_to_replace = list(ach_guide[ach_guide['Leader'].str.contains('Teddy')].index)
        
    
    if row_to_copy in rows_to_replace:
        rows_to_replace.remove(row_to_copy)


    for row in rows_to_replace:
        ach_guide.loc[row, 'achieved'] = ach_guide.loc[row_to_copy, 'achieved']
    
    if pd.isna(ach_guide.loc[row_to_copy, 'Link']):
        ach_guide.drop(row_to_copy, inplace=True)

In [17]:
#ach_guide.to_csv('guide_with_achievements.csv')